## Data Preparation

### Data Preporocessing


In [10]:
# Loading the packages I will need:

print("Loading packages...", end = '')

import numpy as np
from pyunpack import Archive 
import pandas as pd 
import missingno as msno
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error 

import zipfile
import shutil  # For handling directories


print("Successful.")

Loading packages...Successful.


In [11]:
# # Paths
# main_dir = "data"  # Main directory containing year folders
# output_dir = "./temp_extracted_files"  # Temporary directory for extracted files

# # Ensure the output directory exists
# os.makedirs(output_dir, exist_ok=True)

# # List to store dataframes
# dataframes = []

# # Iterate over year folders
# for year in os.listdir(main_dir):
#     year_path = os.path.join(main_dir, year)
#     if os.path.isdir(year_path):  # Ensure it's a folder
#         print(f"Processing year folder: {year}")
        
#         # Process each .zip file within the year folder
#         for month_zip in os.listdir(year_path):
#             month_zip_path = os.path.join(year_path, month_zip)
            
#             if month_zip.endswith(".zip"):
#                 print(f"Extracting: {month_zip}")
#                 # Extract the .zip file
#                 with zipfile.ZipFile(month_zip_path, 'r') as zip_ref:
#                     zip_ref.extractall(output_dir)
                
#                 # Process the extracted files
#                 extracted_files = os.listdir(output_dir)
#                 print(f"Extracted files: {extracted_files}")
                
#                 for extracted_file in extracted_files:
#                     extracted_file_path = os.path.join(output_dir, extracted_file)
                    
#                     # Check for .csv and .xlsx files
#                     if extracted_file.endswith(".csv"):
#                         print(f"Reading CSV file: {extracted_file}")
#                         df = pd.read_csv(extracted_file_path)
#                         df['Year'] = int(year)  # Add a column for the year as integer
#                         dataframes.append(df)
                    
#                     elif extracted_file.endswith((".xlsx", ".xls")):
#                         print(f"Reading Excel file: {extracted_file}")
#                         df = pd.read_excel(extracted_file_path)
#                         df['Year'] = int(year)  # Add a column for the year as integer
#                         dataframes.append(df)
                
#                 # Clean up the temporary directory
#                 for temp_file in extracted_files:
#                     temp_file_path = os.path.join(output_dir, temp_file)
#                     if os.path.isfile(temp_file_path):
#                         os.remove(temp_file_path)  # Remove files
#                     elif os.path.isdir(temp_file_path):
#                         shutil.rmtree(temp_file_path)  # Remove directories

# # Combine dataframes
# if dataframes:
#     combined_df = pd.concat(dataframes, ignore_index=True)
#     combined_file = "combined_data.csv"
#     combined_df.to_csv(combined_file, index=False)
#     print(f"Combined data saved as {combined_file}")
# else:
#     print("No valid files found to combine.")

In [12]:

# Paths
main_dir = "data"  # Main directory containing year folders
output_dir = "./temp_extracted_files"  # Temporary directory for extracted files
chunk_dir = "./chunked_data"  # Directory to save chunked data

# Ensure the output directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(chunk_dir, exist_ok=True)

# Function to process and save chunks
def process_and_save_chunks(file_path, year, chunk_size=100000):
    chunk_list = []
    try:
        for i, chunk in enumerate(pd.read_csv(file_path, encoding='utf-8', chunksize=chunk_size)):
            chunk['Year'] = int(year)
            chunk_file = os.path.join(chunk_dir, f"{os.path.basename(file_path)}_chunk_{i}.csv")
            chunk.to_csv(chunk_file, index=False)
            chunk_list.append(chunk_file)
    except UnicodeDecodeError:
        for i, chunk in enumerate(pd.read_csv(file_path, encoding='latin1', chunksize=chunk_size)):
            chunk['Year'] = int(year)
            chunk_file = os.path.join(chunk_dir, f"{os.path.basename(file_path)}_chunk_{i}.csv")
            chunk.to_csv(chunk_file, index=False)
            chunk_list.append(chunk_file)
    return chunk_list

# Iterate over year folders
for year in os.listdir(main_dir):
    year_path = os.path.join(main_dir, year)
    if os.path.isdir(year_path):  # Ensure it's a folder
        print(f"Processing year folder: {year}")
        
        # Process each .zip file within the year folder
        for month_zip in os.listdir(year_path):
            month_zip_path = os.path.join(year_path, month_zip)
            
            if month_zip.endswith(".zip"):
                print(f"Extracting: {month_zip}")
                # Extract the .zip file
                with zipfile.ZipFile(month_zip_path, 'r') as zip_ref:
                    zip_ref.extractall(output_dir)
                
                # Process the extracted files and directories
                for root, dirs, files in os.walk(output_dir):
                    for file in files:
                        extracted_file_path = os.path.join(root, file)
                        
                        # Check for .csv and .xlsx files
                        if file.endswith(".csv"):
                            print(f"Reading CSV file: {file}")
                            process_and_save_chunks(extracted_file_path, year)
                        
                        elif file.endswith((".xlsx", ".xls")):
                            print(f"Reading Excel file: {file}")
                            df = pd.read_excel(extracted_file_path)
                            df['Year'] = int(year)  # Add a column for the year as integer
                            chunk_file = os.path.join(chunk_dir, f"{os.path.basename(file)}_{year}.csv")
                            df.to_csv(chunk_file, index=False)
                
                # Clean up the temporary directory
                for root, dirs, files in os.walk(output_dir):
                    for file in files:
                        os.remove(os.path.join(root, file))
                    for dir in dirs:
                        shutil.rmtree(os.path.join(root, dir))

# Combine all chunked CSV files into a final combined CSV file
chunk_files = [os.path.join(chunk_dir, f) for f in os.listdir(chunk_dir) if f.endswith(".csv")]
combined_df = pd.concat((pd.read_csv(f) for f in chunk_files), ignore_index=True)
combined_file = "combined_data.csv"
combined_df.to_csv(combined_file, index=False)
print(f"Final combined data saved as {combined_file}")

Processing year folder: 2020
Extracting: April2020TransBorderRawData.zip
Reading CSV file: dot1_0420.csv
Reading CSV file: dot1_ytd_0420.csv
Reading CSV file: dot2_0420.csv
Reading CSV file: dot2_ytd_0420.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_14632\3207700804.py:14: DtypeWarning: Columns (4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(file_path, encoding='utf-8', chunksize=chunk_size)):


Reading CSV file: dot3_0420.csv
Reading CSV file: dot3_ytd_0420.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_14632\3207700804.py:14: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(file_path, encoding='utf-8', chunksize=chunk_size)):


Extracting: August2020TransBorderRawData.zip
Reading CSV file: dot1_0820.csv
Reading CSV file: dot1_ytd_0820.csv
Reading CSV file: dot2_0820.csv


...

C:\Users\DELL\AppData\Local\Temp\ipykernel_14632\3207700804.py:14: DtypeWarning: Columns (4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(file_path, encoding='utf-8', chunksize=chunk_size)):


Reading CSV file: dot2_ytd_0620.csv
Reading CSV file: dot3_0620.csv
Reading CSV file: dot3_ytd_0620.csv
Extracting: March2020TransBorderRawData.zip
Reading CSV file: dot1_0320.csv
Reading CSV file: dot1_ytd_0320.csv
Reading CSV file: dot2_0320.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_14632\3207700804.py:14: DtypeWarning: Columns (4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(file_path, encoding='utf-8', chunksize=chunk_size)):


Reading CSV file: dot2_ytd_0320.csv


KeyboardInterrupt: 

In [5]:
# Display the first five rows to inspect
print(combined_df.head())


  Unnamed: 0  Year  TRDTYPE USASTATE  DEPE  DISAGMOT MEXSTATE CANPROV  \
0        NaN  2020      1.0       AK  07XX       3.0      NaN      XA   
1        NaN  2020      1.0       AK  20XX       3.0      NaN      XA   
2        NaN  2020      1.0       AK  20XX       3.0      NaN      XA   
3        NaN  2020      1.0       AK  20XX       3.0      NaN      XC   
4        NaN  2020      1.0       AK  20XX       3.0      NaN      XM   

   COUNTRY     VALUE  SHIPWT  FREIGHT_CHARGES   DF CONTCODE  MONTH    YEAR  \
0   1220.0    3302.0   378.0            125.0  1.0        X    1.0  2020.0   
1   1220.0  133362.0   137.0           1563.0  1.0        X    1.0  2020.0   
2   1220.0   49960.0    66.0           2631.0  2.0        X    1.0  2020.0   
3   1220.0   21184.0  3418.0            795.0  1.0        X    1.0  2020.0   
4   1220.0    4253.0     2.0             75.0  1.0        X    1.0  2020.0   

   COMMODITY2  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         Na

In [6]:
# Check the columns and their data types
combined_df.dtypes


Unnamed: 0          object
Year                object
TRDTYPE            float64
USASTATE            object
DEPE                object
DISAGMOT           float64
MEXSTATE            object
CANPROV             object
COUNTRY            float64
VALUE              float64
SHIPWT             float64
FREIGHT_CHARGES    float64
DF                 float64
CONTCODE            object
MONTH              float64
YEAR               float64
COMMODITY2         float64
dtype: object

In [7]:
# Display the last five rows of the DataFrame
combined_df.tail()


,Unnamed: 0,Year,TRDTYPE,USASTATE,DEPE,DISAGMOT,MEXSTATE,CANPROV,COUNTRY,VALUE,SHIPWT,FREIGHT_CHARGES,DF,CONTCODE,MONTH,YEAR,COMMODITY2
35391222,NaN,2022,2.0,NaN,55XX,5.0,NaN,NaN,1220.0,3132978.0,4396.0,2857.0,NaN,0,12.0,2022.0,98.0
35391223,NaN,2022,2.0,NaN,60XX,8.0,NaN,NaN,1220.0,14944.0,680.0,150.0,NaN,0,12.0,2022.0,89.0
35391224,NaN,2022,2.0,NaN,60XX,8.0,NaN,NaN,1220.0,5000.0,17.0,450.0,NaN,0,12.0,2022.0,98.0
35391225,NaN,2022,2.0,NaN,70XX,8.0,NaN,NaN,1220.0,207760408.0,0.0,0.0,NaN,0,12.0,2022.0,99.0
35391226,NaN,2022,2.0,NaN,70XX,8.0,NaN,NaN,2010.0,221420268.0,0.0,0.0,NaN,0,12.0,2022.0,99.0


In [8]:
# check first five rows

combined_df.head()

,Unnamed: 0,Year,TRDTYPE,USASTATE,DEPE,DISAGMOT,MEXSTATE,CANPROV,COUNTRY,VALUE,SHIPWT,FREIGHT_CHARGES,DF,CONTCODE,MONTH,YEAR,COMMODITY2
0,NaN,2020,1.0,AK,07XX,3.0,NaN,XA,1220.0,3302.0,378.0,125.0,1.0,X,1.0,2020.0,NaN
1,NaN,2020,1.0,AK,20XX,3.0,NaN,XA,1220.0,133362.0,137.0,1563.0,1.0,X,1.0,2020.0,NaN
2,NaN,2020,1.0,AK,20XX,3.0,NaN,XA,1220.0,49960.0,66.0,2631.0,2.0,X,1.0,2020.0,NaN
3,NaN,2020,1.0,AK,20XX,3.0,NaN,XC,1220.0,21184.0,3418.0,795.0,1.0,X,1.0,2020.0,NaN
4,NaN,2020,1.0,AK,20XX,3.0,NaN,XM,1220.0,4253.0,2.0,75.0,1.0,X,1.0,2020.0,NaN


In [9]:
# Check the total number of rows and columns in the DataFrame
combined_df.shape


(35391227, 17)

In [10]:
# Check unique values in the YEAR column
print("Unique values in YEAR column:")
print(combined_df['YEAR'].unique())

# Check unique values in the Year column
print("Unique values in Year column:")
print(combined_df['Year'].unique())

Unique values in YEAR column:
[2020. 2021. 2022. 2023. 2024.   nan]
Unique values in Year column:
[2020 2021 2022 2023 2024]


Drop the YEAR column

In [1]:

# Paths
combined_file = "combined_data.csv"
processed_file = "processed_combined_data.csv"

# Process the combined CSV file in chunks
chunk_size = 100000
chunks = pd.read_csv(combined_file, chunksize=chunk_size)

# Open a new CSV file to write the processed data
with open(processed_file, 'w', newline='') as f:
    for i, chunk in enumerate(chunks):
        # Drop the YEAR column
        chunk = chunk.drop(columns=['YEAR'])
        
        # Write the chunk to the new CSV file
        if i == 0:
            chunk.to_csv(f, index=False)
        else:
            chunk.to_csv(f, index=False, header=False)

print(f"Processed data saved as {processed_file}")

C:\Users\DELL\AppData\Local\Temp\ipykernel_15952\1041252876.py:13: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
C:\Users\DELL\AppData\Local\Temp\ipykernel_15952\1041252876.py:13: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
C:\Users\DELL\AppData\Local\Temp\ipykernel_15952\1041252876.py:13: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
C:\Users\DELL\AppData\Local\Temp\ipykernel_15952\1041252876.py:13: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
C:\Users\DELL\AppData\Local\Temp\ipykernel_15952\1041252876.py:13: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  

Processed data saved as processed_combined_data.csv


Check unique values in the Year column we maintained

In [4]:
# Path to the processed CSV file
processed_file = "processed_combined_data.csv"

# Read the processed CSV file in smaller chunks
chunk_size = 50000
chunks = pd.read_csv(processed_file, chunksize=chunk_size)

# Initialize variables to store column names and unique values
columns = None
unique_years = set()

# Process each chunk
for chunk in chunks:
    # Get the column names
    if columns is None:
        columns = chunk.columns
    
    # Get the unique values in the Year column
    unique_years.update(chunk['Year'].unique())

# Display the column names
print("Columns in combined_df:")
print(columns)

# Display the unique values in the Year column
print("Unique values in Year column:")
print(sorted(unique_years))

C:\Users\DELL\AppData\Local\Temp\ipykernel_15952\1271478972.py:15: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\DELL\AppData\Local\Temp\ipykernel_15952\1271478972.py:15: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\DELL\AppData\Local\Temp\ipykernel_15952\1271478972.py:15: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\DELL\AppData\Local\Temp\ipykernel_15952\1271478972.py:15: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\DELL\AppData\Local\Temp\ipykernel_15952\1271478972.py:15: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\DELL\AppData\Local\Temp\ipyke

Columns in combined_df:
Index(['Unnamed: 0', 'Year', 'TRDTYPE', 'USASTATE', 'DEPE', 'DISAGMOT',
       'MEXSTATE', 'CANPROV', 'COUNTRY', 'VALUE', 'SHIPWT', 'FREIGHT_CHARGES',
       'DF', 'CONTCODE', 'MONTH', 'COMMODITY2'],
      dtype='object')
Unique values in Year column:
[np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]


Reconnection Point

In [4]:
# Path to the processed CSV file
processed_file = "processed_combined_data.csv"

# Reload the processed CSV file
combined_df = pd.read_csv(processed_file)


C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\332825459.py:5: DtypeWarning: Columns (3,4,6,7,13) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv(processed_file)


In [5]:
combined_df.head()

,Unnamed: 0,Year,TRDTYPE,USASTATE,DEPE,DISAGMOT,MEXSTATE,CANPROV,COUNTRY,VALUE,SHIPWT,FREIGHT_CHARGES,DF,CONTCODE,MONTH,COMMODITY2
0,NaN,2020,1.0,AK,07XX,3.0,NaN,XA,1220.0,3302.0,378.0,125.0,1.0,X,1.0,NaN
1,NaN,2020,1.0,AK,20XX,3.0,NaN,XA,1220.0,133362.0,137.0,1563.0,1.0,X,1.0,NaN
2,NaN,2020,1.0,AK,20XX,3.0,NaN,XA,1220.0,49960.0,66.0,2631.0,2.0,X,1.0,NaN
3,NaN,2020,1.0,AK,20XX,3.0,NaN,XC,1220.0,21184.0,3418.0,795.0,1.0,X,1.0,NaN
4,NaN,2020,1.0,AK,20XX,3.0,NaN,XM,1220.0,4253.0,2.0,75.0,1.0,X,1.0,NaN


Remove the 'Unnamed: 0' column: This column is likely an artifact from saving and loading the DataFrame.

In [6]:

# Remove the 'Unnamed: 0' column
combined_df = combined_df.drop(columns=['Unnamed: 0'])

# Verify the changes
print(combined_df.head())
print("Columns in combined_df:")
print(combined_df.columns)

   Year  TRDTYPE USASTATE  DEPE  DISAGMOT MEXSTATE CANPROV  COUNTRY     VALUE  \
0  2020      1.0       AK  07XX       3.0      NaN      XA   1220.0    3302.0   
1  2020      1.0       AK  20XX       3.0      NaN      XA   1220.0  133362.0   
2  2020      1.0       AK  20XX       3.0      NaN      XA   1220.0   49960.0   
3  2020      1.0       AK  20XX       3.0      NaN      XC   1220.0   21184.0   
4  2020      1.0       AK  20XX       3.0      NaN      XM   1220.0    4253.0   

   SHIPWT  FREIGHT_CHARGES   DF CONTCODE  MONTH  COMMODITY2  
0   378.0            125.0  1.0        X    1.0         NaN  
1   137.0           1563.0  1.0        X    1.0         NaN  
2    66.0           2631.0  2.0        X    1.0         NaN  
3  3418.0            795.0  1.0        X    1.0         NaN  
4     2.0             75.0  1.0        X    1.0         NaN  
Columns in combined_df:
Index(['Year', 'TRDTYPE', 'USASTATE', 'DEPE', 'DISAGMOT', 'MEXSTATE',
       'CANPROV', 'COUNTRY', 'VALUE', 'SHIPWT',

In [7]:

# Check the updated DataFrame
combined_df.head(10)


,Year,TRDTYPE,USASTATE,DEPE,DISAGMOT,MEXSTATE,CANPROV,COUNTRY,VALUE,SHIPWT,FREIGHT_CHARGES,DF,CONTCODE,MONTH,COMMODITY2
0,2020,1.0,AK,07XX,3.0,NaN,XA,1220.0,3302.0,378.0,125.0,1.0,X,1.0,NaN
1,2020,1.0,AK,20XX,3.0,NaN,XA,1220.0,133362.0,137.0,1563.0,1.0,X,1.0,NaN
2,2020,1.0,AK,20XX,3.0,NaN,XA,1220.0,49960.0,66.0,2631.0,2.0,X,1.0,NaN
3,2020,1.0,AK,20XX,3.0,NaN,XC,1220.0,21184.0,3418.0,795.0,1.0,X,1.0,NaN
4,2020,1.0,AK,20XX,3.0,NaN,XM,1220.0,4253.0,2.0,75.0,1.0,X,1.0,NaN
5,2020,1.0,AK,20XX,3.0,NaN,XO,1220.0,26587.0,413.0,555.0,1.0,X,1.0,NaN
6,2020,1.0,AK,20XX,3.0,NaN,XO,1220.0,128192.0,5193.0,266.0,2.0,X,1.0,NaN
7,2020,1.0,AK,20XX,3.0,NaN,XQ,1220.0,4597.0,526.0,482.0,1.0,X,1.0,NaN
8,2020,1.0,AK,20XX,3.0,XX,NaN,2010.0,2970.0,1.0,0.0,1.0,0,1.0,NaN
9,2020,1.0,AK,2304,5.0,CM,NaN,2010.0,125986.0,0.0,0.0,1.0,0,1.0,NaN


Data Cleaning

In [8]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35391227 entries, 0 to 35391226
Data columns (total 15 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Year             int64  
 1   TRDTYPE          float64
 2   USASTATE         object 
 3   DEPE             object 
 4   DISAGMOT         float64
 5   MEXSTATE         object 
 6   CANPROV          object 
 7   COUNTRY          float64
 8   VALUE            float64
 9   SHIPWT           float64
 10  FREIGHT_CHARGES  float64
 11  DF               float64
 12  CONTCODE         object 
 13  MONTH            float64
 14  COMMODITY2       float64
dtypes: float64(9), int64(1), object(5)
memory usage: 4.0+ GB


In [9]:
# Check for missing values in each column
missing_values = combined_df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
Year                      0
TRDTYPE                   0
USASTATE            5062252
DEPE               22214024
DISAGMOT                  0
MEXSTATE           25512467
CANPROV            16914390
COUNTRY                   0
VALUE                     0
SHIPWT                    0
FREIGHT_CHARGES           0
DF                 11888913
CONTCODE                  0
MONTH                496662
COMMODITY2          8114951
dtype: int64


In [11]:
combined_df.shape

(35391227, 15)

In [17]:
import pandas as pd

# Path to the processed CSV file
processed_file = "processed_combined_data.csv"
output_file = "processed_combined_data_filled.csv"

# Read the processed CSV file in chunks
chunk_size = 100000
chunks = pd.read_csv(processed_file, chunksize=chunk_size)

# Open a new CSV file to write the processed data
with open(output_file, 'w', newline='') as f:
    for i, chunk in enumerate(chunks):
        # Fill missing values in numerical columns with the mean
        chunk['COMMODITY2'] = chunk['COMMODITY2'].fillna(chunk['COMMODITY2'].mean())
        chunk['DF'] = chunk['DF'].fillna(chunk['DF'].mean())

        # Fill missing values in categorical columns with a specific value or mode
        chunk['USASTATE'] = chunk['USASTATE'].fillna('Unknown')
        chunk['DEPE'] = chunk['DEPE'].fillna('Unknown')
        chunk['MEXSTATE'] = chunk['MEXSTATE'].fillna('Unknown')
        chunk['CANPROV'] = chunk['CANPROV'].fillna('Unknown')

        # Fill missing values in the MONTH column with the mode if it exists and has non-empty values
        if 'MONTH' in chunk.columns and not chunk['MONTH'].isnull().all():
            month_mode = chunk['MONTH'].mode()[0]
            chunk['MONTH'] = chunk['MONTH'].fillna(month_mode)

        # Write the chunk to the new CSV file
        if i == 0:
            chunk.to_csv(f, index=False)
        else:
            chunk.to_csv(f, index=False, header=False)

print(f"Processed data with filled missing values saved as {output_file}")

C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\1893606638.py:13: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\1893606638.py:13: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\1893606638.py:13: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\1893606638.py:13: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\1893606638.py:13: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  

Processed data with filled missing values saved as processed_combined_data_filled.csv


In [20]:
# Path to the processed CSV file with filled missing values
input_file = "processed_combined_data_filled.csv"
output_file = "processed_combined_data_transformed.csv"

# Read the processed CSV file in chunks
chunk_size = 100000
chunks = pd.read_csv(input_file, chunksize=chunk_size)

# Open a new CSV file to write the processed data
with open(output_file, 'w', newline='') as f:
    for i, chunk in enumerate(chunks):
        # Change the dtype of TRDTYPE from float to string and replace values
        chunk['TRDTYPE'] = chunk['TRDTYPE'].astype(str).replace({'1.0': 'Export', '2.0': 'Import'})

        # Change COMMODITY2 from float to int, handle non-numeric values
        chunk['COMMODITY2'] = pd.to_numeric(chunk['COMMODITY2'], errors='coerce').fillna(0).astype(int)

        # Replace values in CONTCODE
        chunk['CONTCODE'] = chunk['CONTCODE'].replace({'X': 'Containerized', '0': 'Non-Containerized'})

        # Handle NaN values in DF, change DF from float to int and then to string, replace values
        chunk['DF'] = chunk['DF'].fillna(0).astype(int).astype(str).replace({'1': 'domestic merchandise', '2': 'foreign merchandise'})

        # Replace values in COUNTRY
        chunk['COUNTRY'] = chunk['COUNTRY'].astype(str).replace({'1220.0': 'Canada', '2010.0': 'Mexico'})

        # Change DISAGMOT from float to string and replace values
        chunk['DISAGMOT'] = chunk['DISAGMOT'].astype(str).replace({
            '1.0': 'Vessel', '3.0': 'Air', '4.0': 'Mail (U.S. Postal Service)', '5.0': 'Truck',
            '6.0': 'Rail', '7.0': 'Pipeline', '8.0': 'Other', '9.0': 'Foreign Trade Zones (FTZs)'
        })

        # Change MONTH from float to string and replace values
        chunk['MONTH'] = chunk['MONTH'].astype(str).replace({
            '1.0': 'January', '2.0': 'February', '3.0': 'March', '4.0': 'April', '5.0': 'May', '6.0': 'June',
            '7.0': 'July', '8.0': 'August', '9.0': 'September', '10.0': 'October', '11.0': 'November', '12.0': 'December'
        })

        # Change YEAR from int to datetime
        chunk['Year'] = pd.to_datetime(chunk['Year'], format='%Y')

        # Change VALUE from float to int
        chunk['VALUE'] = chunk['VALUE'].astype(int)

        # Change FREIGHT_CHARGES from float to int
        chunk['FREIGHT_CHARGES'] = chunk['FREIGHT_CHARGES'].astype(int)

        # Change SHIPWT from float to int
        chunk['SHIPWT'] = chunk['SHIPWT'].astype(int)

        # Write the chunk to the new CSV file
        if i == 0:
            chunk.to_csv(f, index=False)
        else:
            chunk.to_csv(f, index=False, header=False)

print(f"Processed data with transformations saved as {output_file}")

C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\3729632202.py:11: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\3729632202.py:11: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\3729632202.py:11: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\3729632202.py:11: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\3729632202.py:11: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chu

Processed data with transformations saved as processed_combined_data_transformed.csv


In [ ]:

# Path to the transformed CSV file
output_file = "processed_combined_data_transformed.csv"

# Read the transformed CSV file in chunks
chunk_size = 100000
chunks = pd.read_csv(output_file, chunksize=chunk_size)

# Initialize variables to store column names and data types
columns = None
dtypes = None

# Process each chunk
for chunk in chunks:
    # Get the column names and data types
    if columns is None:
        columns = chunk.columns
        dtypes = chunk.dtypes

    # Display the first few rows of the first chunk to inspect
    if chunk.index[0] == 0:
        print(chunk.head())

# Display the column names and data types
print("Columns in transformed_df:")
print(columns)
print("\nData types in transformed_df:")
print(dtypes)

C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\4110430967.py:15: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:


   Unnamed: 0        Year TRDTYPE USASTATE  DEPE DISAGMOT MEXSTATE CANPROV  \
0         NaN  2020-01-01  Export       AK  07XX      Air  Unknown      XA   
1         NaN  2020-01-01  Export       AK  20XX      Air  Unknown      XA   
2         NaN  2020-01-01  Export       AK  20XX      Air  Unknown      XA   
3         NaN  2020-01-01  Export       AK  20XX      Air  Unknown      XC   
4         NaN  2020-01-01  Export       AK  20XX      Air  Unknown      XM   

  COUNTRY   VALUE  SHIPWT  FREIGHT_CHARGES                    DF  \
0  Canada    3302     378              125  domestic merchandise   
1  Canada  133362     137             1563  domestic merchandise   
2  Canada   49960      66             2631   foreign merchandise   
3  Canada   21184    3418              795  domestic merchandise   
4  Canada    4253       2               75  domestic merchandise   

        CONTCODE    MONTH  COMMODITY2  
0  Containerized  January           0  
1  Containerized  January           0  
2 

C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\4110430967.py:15: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\4110430967.py:15: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\4110430967.py:15: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\4110430967.py:15: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\DELL\AppData\Local\Temp\ipykernel_11716\4110430967.py:15: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
C:\Users\DELL\AppData\Local\Temp\ipykernel_11

Columns in transformed_df:
Index(['Unnamed: 0', 'Year', 'TRDTYPE', 'USASTATE', 'DEPE', 'DISAGMOT',
       'MEXSTATE', 'CANPROV', 'COUNTRY', 'VALUE', 'SHIPWT', 'FREIGHT_CHARGES',
       'DF', 'CONTCODE', 'MONTH', 'COMMODITY2'],
      dtype='object')

Data types in transformed_df:
Unnamed: 0         float64
Year                object
TRDTYPE             object
USASTATE            object
DEPE                object
DISAGMOT            object
MEXSTATE            object
CANPROV             object
COUNTRY             object
VALUE                int64
SHIPWT               int64
FREIGHT_CHARGES      int64
DF                  object
CONTCODE            object
MONTH               object
COMMODITY2           int64
dtype: object


Reconnection Point

To efficiently load and inspect the data in chunks while still being able to work with the DataFrame as transformed_df, I can process each chunk and perform operations incrementally. However, if I need to work with the entire DataFrame, I can save the processed chunks to a new CSV file and then load the smaller chunks as needed.

Here's how I can do it:

1. Inspect the data in chunks: This will allow me to quickly verify the data without loading the entire dataset into memory.
2. Save the processed chunks to a new CSV file: This will allow me to load smaller chunks as needed.
3. Load the processed chunks as needed

I need to read and inspect the reduced data in chunks without loading the entire DataFrame into memory at once.


Step 1: Inspect the data in chunks

In [6]:

# Path to the transformed CSV file
output_file = "processed_combined_data_transformed.csv"

# Read the transformed CSV file in chunks using the 'python' engine
chunk_size = 100000
chunks = pd.read_csv(output_file, chunksize=chunk_size, engine='python')

# Process the first chunk to inspect the data
for i, chunk in enumerate(chunks):
    if i == 0:
        # Display the DataFrame information for the first chunk
        chunk.info()
        
        # Display the first few rows of the first chunk to inspect
        print(chunk.head())
        
        # Break after processing the first chunk
        break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       0 non-null       float64
 1   Year             100000 non-null  object 
 2   TRDTYPE          100000 non-null  object 
 3   USASTATE         100000 non-null  object 
 4   DEPE             100000 non-null  object 
 5   DISAGMOT         100000 non-null  object 
 6   MEXSTATE         100000 non-null  object 
 7   CANPROV          100000 non-null  object 
 8   COUNTRY          100000 non-null  object 
 9   VALUE            100000 non-null  int64  
 10  SHIPWT           100000 non-null  int64  
 11  FREIGHT_CHARGES  100000 non-null  int64  
 12  DF               100000 non-null  object 
 13  CONTCODE         100000 non-null  object 
 14  MONTH            100000 non-null  object 
 15  COMMODITY2       100000 non-null  int64  
dtypes: float64(1), int64(4), object(11)
mem

Step 2: Save the processed chunks to a new CSV file

In [7]:
# Path to the transformed CSV file
output_file = "processed_combined_data_transformed.csv"
new_output_file = "processed_combined_data_transformed_chunks.csv"

# Read the transformed CSV file in chunks using the 'python' engine
chunk_size = 100000
chunks = pd.read_csv(output_file, chunksize=chunk_size, engine='python')

# Open a new CSV file to write the processed data
with open(new_output_file, 'w', newline='') as f:
    for i, chunk in enumerate(chunks):
        # Write the chunk to the new CSV file
        if i == 0:
            chunk.to_csv(f, index=False)
        else:
            chunk.to_csv(f, index=False, header=False)

print(f"Processed data saved as {new_output_file}")

Processed data saved as processed_combined_data_transformed_chunks.csv


Step 3: Load the processed chunks as needed

In [13]:

# Path to the transformed CSV file
input_file = "processed_combined_data_transformed.csv"
output_file = "processed_combined_data_reduced.csv"

# Number of rows to drop for each year
rows_to_drop_per_year = 4000000

# Read the transformed CSV file in chunks using the 'python' engine
chunk_size = 100000
chunks = pd.read_csv(input_file, chunksize=chunk_size, engine='python')

# Initialize dictionaries to keep track of rows to drop for each year
rows_to_drop = {2020: rows_to_drop_per_year, 2021: rows_to_drop_per_year, 2022: rows_to_drop_per_year, 2023: rows_to_drop_per_year, 2024: rows_to_drop_per_year}

# Open a new CSV file to write the remaining data
with open(output_file, 'w', newline='') as f:
    for i, chunk in enumerate(chunks):
        # Filter and sample rows for each year
        for year in rows_to_drop.keys():
            year_rows = chunk[chunk['Year'] == year]
            if not year_rows.empty:
                if rows_to_drop[year] > 0:
                    sample_size = min(rows_to_drop[year], len(year_rows))
                    drop_indices = np.random.choice(year_rows.index, sample_size, replace=False)
                    chunk = chunk.drop(drop_indices)
                    rows_to_drop[year] -= sample_size

        # Write the remaining rows to the new CSV file
        if i == 0:
            chunk.to_csv(f, index=False)
        else:
            chunk.to_csv(f, index=False, header=False)

print(f"Processed data with reduced rows saved as {output_file}")

# Read and inspect the reduced data in chunks
chunk_size = 100000
chunks = pd.read_csv(output_file, chunksize=chunk_size, engine='python')

# Process the first chunk to inspect the data
for i, chunk in enumerate(chunks):
    if i == 0:
        # Display the DataFrame information for the first chunk
        chunk.info()
        
        # Display the first few rows of the first chunk to inspect
        print(chunk.head())
        
        # Break after processing the first chunk
        break

Processed data with reduced rows saved as processed_combined_data_reduced.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       0 non-null       float64
 1   Year             100000 non-null  object 
 2   TRDTYPE          100000 non-null  object 
 3   USASTATE         100000 non-null  object 
 4   DEPE             100000 non-null  object 
 5   DISAGMOT         100000 non-null  object 
 6   MEXSTATE         100000 non-null  object 
 7   CANPROV          100000 non-null  object 
 8   COUNTRY          100000 non-null  object 
 9   VALUE            100000 non-null  int64  
 10  SHIPWT           100000 non-null  int64  
 11  FREIGHT_CHARGES  100000 non-null  int64  
 12  DF               100000 non-null  object 
 13  CONTCODE         100000 non-null  object 
 14  MONTH            100000 non-null  object 
 15  COMMODIT

In [ ]:
# Path to the new CSV file with reduced data
output_file = "processed_combined_data_reduced.csv"

# Load the reduced data into a DataFrame called reduced_df
reduced_df = pd.read_csv(output_file, engine='python')

# Display the DataFrame information
reduced_df.info()

# Display the first few rows to inspect
print(reduced_df.head())

lets remove the unnamed: 0 column and why is the Year column in string instead of datetime

#### change for some dtypes
COMMODITY2 from float to int

Container Code (CONTCODE) Code Containerized?  
X Containerized 
0 Non-Containerized 
 
  DF from float to int

  Country Code (COUNTRY)  float to string
Code Country 
1220 Canada 
2010 Mexico 
 
Domestic/Foreign Code (DF)   float to string 
Distinguishes whether the merchandise was produced in the U.S. 
Code 
1 domestically produced merchandise 
2 foreign produced merchandise  


Mode of Transportation Code (DISAGMOT)  float to string 
Code Description 
1 Vessel 
3 Air 
4 Mail (U.S. Postal Service) 
5 Truck 
6 Rail 
7 Pipeline 
8 Other 
9 Foreign Trade Zones (FTZs) 


Month (MONTH)    float to string
Month 
Code Month 
  1 January 
  2 February 
  3 March 
  4 April 
  5 May 
  6 June 
  7 July 
  8 August 
  9 September 
10 October 
11 November 
12 December



Trade Type Code (TRDTYPE)     float to string
Code Trade Type 
1 Export 
2 Import  



Value (VALUE)    float to int
Value of Goods in United States Dollars 
 
Year (YEAR)  int to datetime


Freight Charges (FREIGHT_CHARGES)   float to int 
Freight Charges in U.S. Dollars


MONTH  int to date time 

SHIPWT float to int


In [26]:
combined_df.describe()

,TRDTYPE,DISAGMOT,COUNTRY,VALUE,SHIPWT,FREIGHT_CHARGES,DF,MONTH,YEAR,COMMODITY2
count,2.051251e+06,2.051251e+06,2.051251e+06,2.051251e+06,2.051251e+06,2.051251e+06,1.357619e+06,2.051251e+06,2.051251e+06,1.580717e+06
mean,1.338151e+00,4.756261e+00,1.528714e+03,2.390849e+06,1.209763e+06,3.337063e+04,1.325863e+00,5.091202e+00,2.020663e+03,5.652508e+01
std,4.730802e-01,1.251327e+00,3.854605e+02,3.027425e+07,3.780300e+07,8.343282e+05,4.686966e-01,2.895490e+00,1.426718e+00,2.792732e+01
min,1.000000e+00,1.000000e+00,1.220000e+03,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,2.020000e+03,1.000000e+00
25%,1.000000e+00,5.000000e+00,1.220000e+03,1.355500e+04,0.000000e+00,0.000000e+00,1.000000e+00,3.000000e+00,2.020000e+03,3.300000e+01
50%,1.000000e+00,5.000000e+00,1.220000e+03,6.490400e+04,0.000000e+00,2.840000e+02,1.000000e+00,5.000000e+00,2.020000e+03,5.900000e+01
75%,2.000000e+00,5.000000e+00,2.010000e+03,3.778740e+05,2.624000e+03,2.847000e+03,2.000000e+00,7.000000e+00,2.020000e+03,8.400000e+01
max,2.000000e+00,9.000000e+00,2.010000e+03,5.595625e+09,8.110661e+09,2.046194e+08,2.000000e+00,1.200000e+01,2.024000e+03,9.900000e+01


In [28]:
# Check unique values for each column
for column in combined_df.columns:
    unique_values = combined_df[column].unique()
    print(f"Unique values in column {column}:")
    print(unique_values)
    print("\n")

Unique values in column TRDTYPE:
['Export' 'Import']


Unique values in column USASTATE:
['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'DU' 'FL' 'GA' 'HI' 'IA'
 'ID' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT'
 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC'
 'SD' 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY' nan]


Unique values in column DEPE:
['0712' '20XX' '2301' '2303' '2304' '2305' '2401' '2506' '2604' '3001'
 '3004' '3009' '30XX' '3101' '3103' '3104' '3105' '3106' '3111' '3126'
 '31XX' '3310' '34XX' '3801' '3802' '4101' '41XX' '70XX' '0104' '0106'
 '0115' '04XX' '0701' '0704' '0708' '07XX' '0901' '09XX' '17XX' '18XX'
 '19XX' '2006' '2402' '2404' '2503' '2507' '2601' '2603' '2608' '3019'
 '3023' '3302' '3401' '3403' '3501' '35XX' '3604' '3803' '3804' '38XX'
 '4102' '4115' '5203' '52XX' '53XX' '55XX' '0209' '2403' '2408' '3003'
 '33XX' '3613' '0212' '11XX' '15XX' '2307' '2310' '24XX' '2501' '2602'
 '2605' '2606' '2609' '26XX' '2720' '2801'

In [30]:
# Check unique values for each column
for column in combined_df.columns:
    unique_values = combined_df[column].unique()
    print(f"Unique values in column {column}:")
    print(unique_values)
    print("\n")

Unique values in column TRDTYPE:
['Export' 'Import']


Unique values in column USASTATE:
['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'DU' 'FL' 'GA' 'HI' 'IA'
 'ID' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT'
 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC'
 'SD' 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY' nan]


Unique values in column DEPE:
['0712' '20XX' '2301' '2303' '2304' '2305' '2401' '2506' '2604' '3001'
 '3004' '3009' '30XX' '3101' '3103' '3104' '3105' '3106' '3111' '3126'
 '31XX' '3310' '34XX' '3801' '3802' '4101' '41XX' '70XX' '0104' '0106'
 '0115' '04XX' '0701' '0704' '0708' '07XX' '0901' '09XX' '17XX' '18XX'
 '19XX' '2006' '2402' '2404' '2503' '2507' '2601' '2603' '2608' '3019'
 '3023' '3302' '3401' '3403' '3501' '35XX' '3604' '3803' '3804' '38XX'
 '4102' '4115' '5203' '52XX' '53XX' '55XX' '0209' '2403' '2408' '3003'
 '33XX' '3613' '0212' '11XX' '15XX' '2307' '2310' '24XX' '2501' '2602'
 '2605' '2606' '2609' '26XX' '2720' '2801'

In [31]:
# Change the dtype of TRDTYPE from int to string and replace values
combined_df['TRDTYPE'] = combined_df['TRDTYPE'].astype(str).replace({'1': 'Export', '2': 'Import'})

# Change COMMODITY2 from float to int, handle non-numeric values
combined_df['COMMODITY2'] = pd.to_numeric(combined_df['COMMODITY2'], errors='coerce').fillna(0).astype(int)

# Replace values in CONTCODE
combined_df['CONTCODE'] = combined_df['CONTCODE'].replace({'X': 'Containerized', '0': 'Non-Containerized'})

# Handle NaN values in DF, change DF from float to int and then to string, replace values
combined_df['DF'] = combined_df['DF'].fillna(0).astype(int).astype(str).replace({'1': 'domestic merchandise', '2': 'foreign merchandise'})

# Replace values in COUNTRY
combined_df['COUNTRY'] = combined_df['COUNTRY'].replace({1220: 'Canada', 2010: 'Mexico'})

# Change DISAGMOT from int to string and replace values
combined_df['DISAGMOT'] = combined_df['DISAGMOT'].astype(str).replace({
    '1': 'Vessel', '3': 'Air', '4': 'Mail (U.S. Postal Service)', '5': 'Truck',
    '6': 'Rail', '7': 'Pipeline', '8': 'Other', '9': 'Foreign Trade Zones (FTZs)'
})

# Change MONTH from int to string and replace values
combined_df['MONTH'] = combined_df['MONTH'].astype(str).replace({
    '1': 'January', '2': 'February', '3': 'March', '4': 'April', '5': 'May', '6': 'June',
    '7': 'July', '8': 'August', '9': 'September', '10': 'October', '11': 'November', '12': 'December'
})

# Change YEAR from int to datetime
combined_df['YEAR'] = pd.to_datetime(combined_df['YEAR'], format='%Y')

# Verify the changes
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051251 entries, 0 to 2051250
Data columns (total 15 columns):
 #   Column           Dtype         
---  ------           -----         
 0   TRDTYPE          object        
 1   USASTATE         object        
 2   DEPE             object        
 3   DISAGMOT         object        
 4   MEXSTATE         object        
 5   CANPROV          object        
 6   COUNTRY          object        
 7   VALUE            int64         
 8   SHIPWT           int64         
 9   FREIGHT_CHARGES  int64         
 10  DF               object        
 11  CONTCODE         object        
 12  MONTH            object        
 13  YEAR             datetime64[ns]
 14  COMMODITY2       int64         
dtypes: datetime64[ns](1), int64(4), object(10)
memory usage: 234.7+ MB


In [32]:
# Check unique values in the YEAR column
print("Unique values in YEAR column:")
print(combined_df['YEAR'].unique())

Unique values in YEAR column:
<DatetimeArray>
['2020-01-01 00:00:00', '2023-01-01 00:00:00', '2024-01-01 00:00:00']
Length: 3, dtype: datetime64[ns]


In [33]:
import os

main_dir = "data"  # Main directory containing year folders

# List the contents of the main directory
print(f"Contents of {main_dir}:")
for year in os.listdir(main_dir):
    year_path = os.path.join(main_dir, year)
    if os.path.isdir(year_path):
        print(f"Year folder: {year}")
        # List the contents of each year folder
        for month_zip in os.listdir(year_path):
            print(f"  {month_zip}")

Contents of data:
Year folder: 2020
  April2020TransBorderRawData.zip
  August2020TransBorderRawData.zip
  February2020TransBorderRawData.zip
  January2020TransBorderRawData.zip
  July2020TransBorderRawData.zip
  June2020TransBorderRawData.zip
  March2020TransBorderRawData.zip
  May2020TransBorderRawData.zip
  September2020TransBorderRawData.zip
Year folder: 2021
  April2021TransBorderRawData.zip
  February2021TransBorderRawData.zip
  January2021TransBorderRawData.zip
  July-to-Dec-2021.zip
  July2021TransBorderRawData.zip
  June2021TransBorderRawData.zip
  March2021TransBorderRawData.zip
  May2021TransBorderRawData.zip
Year folder: 2022
  April2022.zip
  August2022.zip
  December2022.zip
  Feb-2022.zip
  Jan-2022.zip
  July2022.zip
  June2022.zip
  March-2022.zip
  May2022.zip
  November2022.zip
  October2022.zip
  September2022.zip
Year folder: 2023
  April2023.zip
  Aug2023.zip
  December2023.zip
  Feb2023.zip
  Jan2023.zip
  July2023.zip
  June2023.zip
  March2023.zip
  May2023.zip